In [ ]:
from envs.edit_photo import PhotoEditor
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
with open('dataset/trainTarget.txt') as f :
    tif_images = f.read()
    jpg_images = tif_images.replace('tif','jpg')
with open('dataset/trainTarget_jpg.txt','w+') as f:
    f.write(jpg_images)
    
with open('dataset/trainSource.txt') as f :
    tif_images = f.read()
    jpg_images = tif_images.replace('tif','jpg')
with open('dataset/trainSource_jpg.txt','w+') as f:
    f.write(jpg_images)

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609_original.jpg")
target = cv2.imread("sample_images/a0676-kme_609_C.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)/255.0
target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)/255.0
photo_editor = PhotoEditor()
parameters = np.array([0.125, 0.125, 0.375, 0.125, 0., 0.0625, 0.9375, 0.375, 0.0625, 0., 0.125, 0.125])

In [ ]:
image.shape

In [ ]:
plt.imshow(image)

In [ ]:
# from PIL import Image

# def resize_length(image_array, size=512):
#     """
#     Resize the longer side of the image to the specified size while maintaining the aspect ratio.

#     :param image_array: NumPy array representing the image.
#     :param size: The target size for the longer side of the image.
#     :return: Resized image as a NumPy array.
#     """
#     image = Image.fromarray(image_array)
#     original_width, original_height = image.size
#     if original_width > original_height:
#         new_width = size
#         new_height = int((original_height / original_width) * size)
#     else:
#         new_height = size
#         new_width = int((original_width / original_height) * size)
#     resized_image = image.resize((new_width, new_height), Image.LANCZOS)
#     resized_image_array = np.array(resized_image)
    
#     return resized_image_array

In [ ]:
# input = resize_length(image)
# input = input / 255.0

In [ ]:
output = photo_editor(image.copy(),parameters.copy())

In [ ]:
plt.imshow(output)

In [ ]:
import torch
enhanced = torch.Tensor(output)
enhanced_target = torch.Tensor(target)
enhanced =torch.flatten(enhanced.clone(),start_dim=0, end_dim=-1)
enhanced_target = torch.flatten(enhanced_target.clone(),start_dim=0, end_dim=-1)
t= enhanced_target.clone()
t[0]=0
rmse =  enhanced-enhanced_target
rmse = torch.pow(rmse,2).mean()
rmse = torch.sqrt(rmse)

rewards = -rmse
psnr = 20 * torch.log10(1/ rmse) 
psnr = psnr -100
psnr

In [ ]:
torch.Tensor([True,False]).all()

In [ ]:
import torch
from torchrl.data import TensorDictReplayBuffer,LazyMemmapStorage
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from tensordict import TensorDict

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define buffer parameters
buffer_size = 9000  # example buffer size

# Initialize the replay buffer on the GPU
replay_buffer = TensorDictReplayBuffer(
                        storage=LazyMemmapStorage(buffer_size,), sampler=SamplerWithoutReplacement()
                                                        )

# Populate the buffer with sample data
# Example of how to add data to the buffer
for _ in range(buffer_size):
    # Create a sample data point
    state = torch.randn((32,512,), device=device)  # Replace state_size with actual size
    action = torch.randn((32,15,), device=device).clamp(-1, 1)  # 15-dimensional continuous action
    reward = torch.randn(32, device=device)
    next_state = torch.randn((32,512,), device=device)  # Replace state_size with actual size
    done = torch.randn(32, device=device)  # Example done flag
    batch_transition = TensorDict(
            {
                "observations":state.clone(),
                "next_observations":next_state.clone(),
                "actions":action.clone(),
                "rewards":reward.clone(),
                "dones":done.clone(),
            },
            batch_size = 32,
        )
    # Add to the replay buffer
    replay_buffer.add(batch_transition)

# Verify buffer storage on GPU
sample = replay_buffer.sample(1)  # Sample a single transition
print(sample['state'].device)  # Should output: cuda:0
print(sample['action'].device)  # Should output: cuda:0
print(sample['reward'].device)  # Should output: cuda:0
print(sample['next_state'].device)  # Should output: cuda:0
print(sample['done'].device)  #

In [ ]:
from torcheval.metrics import PeakSignalNoiseRatio
metric = PeakSignalNoiseRatio()
metric.update(enhanced.permute(2,0,1), enhanced_target.permute(2,0,1))
metric.compute()

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
import torchvision.transforms.v2.functional as F
import os
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import BertTokenizer, BertModel, CLIPProcessor, CLIPModel
from tqdm import tqdm
import kornia.color as K
class FiveKDataset(Dataset):
    def __init__(self, image_size, mode="train", resize=True, 
                 augment_data=False, use_txt_features=False, device='cuda',
                 pre_load_images=True):
        current_dir = os.getcwd()
        dataset_dir = os.path.join(current_dir,"..","dataset")
        self.IMGS_PATH = os.path.join(dataset_dir, f"FiveK/{mode}")
        self.FEATURES_PATH = os.path.join(dataset_dir, "processed_categories_2.txt")
        self.resize = resize
        self.image_size = image_size
        self.augment_data = augment_data
        self.use_txt_features = use_txt_features
        self.device = device
        self.pre_load_images = pre_load_images
        self.feature_categories = ["Location", "Time", "Light", "Subject"]
        
        # Load semantic features from processed_categories.txt
        self.features = {}
        with open(self.FEATURES_PATH, 'r') as f:
            for line in f:
                parts = line.strip().split(',')
                img_name = parts[0]
                img_features = parts[1:]
                self.features[img_name] = img_features
        
        # Load image files
        self.img_files = [f for f in os.listdir(os.path.join(self.IMGS_PATH, 'input'))]
        
        # Prepare MultiLabelBinarizer
        all_features = [self.features[img] for img in self.img_files]
        self.mlb = MultiLabelBinarizer()
        self.mlb.fit(all_features)
        
        # Create encoding dictionaries for categorical approach
        unique_features = {cat: set(feat[i] for feat in all_features) 
                           for i, cat in enumerate(self.feature_categories)}
        self.feature_to_idx = {
            cat: {feat: idx for idx, feat in enumerate(sorted(features))}
            for cat, features in unique_features.items()
        }
        if self.use_txt_features == "histogram":
            self.histogram_bins = 64       
        # Preload all images and features if pre_load_images is True
        if self.pre_load_images:
            self.preload_data()
        
        if self.use_txt_features == "embedded":
            self.precompute_features()

    def compute_histogram(self, image):
        # Convert RGB to CIE Lab
        lab_image = K.rgb_to_lab(image.unsqueeze(0)/255.0).squeeze(0)
        
        # Compute histogram for each channel
        histograms = []
        for channel in range(3):  # L, a, b channels
            if channel == 0:  # L channel
                hist = torch.histc(lab_image[channel], bins=self.histogram_bins, min=0, max=100)
            else:  # a and b channels
                hist = torch.histc(lab_image[channel], bins=self.histogram_bins, min=-128, max=127)
            # Normalize the histogram
            hist = hist / hist.sum()
            histograms.append(hist)
        
        # Concatenate histograms
        return torch.cat(histograms)
    
    def preload_data(self):
        print("Preloading images and features...")
        self.source_images = []
        self.target_images = []
        self.one_hot_features = []
        self.cat_features = []
        self.histograms = []
        
        for img_name in tqdm(self.img_files):
            # Load and preprocess images
            source = read_image(os.path.join(self.IMGS_PATH, 'input', img_name))
            target = read_image(os.path.join(self.IMGS_PATH, 'target', img_name))
            
            if self.resize:
                source = F.resize(source, (self.image_size, self.image_size), interpolation=F.InterpolationMode.BICUBIC)
                target = F.resize(target, (self.image_size, self.image_size), interpolation=F.InterpolationMode.BICUBIC)
            
            self.source_images.append(source.to(self.device))
            self.target_images.append(target.to(self.device))
            
            # Precompute features
            if self.use_txt_features == "one_hot":
                one_hot = self.mlb.transform([self.features[img_name]])[0]
                self.one_hot_features.append(torch.tensor(one_hot, dtype=torch.float32, device=self.device))
            elif self.use_txt_features == "categorical":
                cat = [self.feature_to_idx[cat][feat] for cat, feat in zip(self.feature_categories, self.features[img_name])]
                self.cat_features.append(torch.tensor(cat, dtype=torch.long, device=self.device))
            elif self.use_txt_features == "histogram":
                source_hist = self.compute_histogram(source).to(self.device)
                target_hist = self.compute_histogram(target).to(self.device)
                self.histograms.append((source_hist, target_hist))
        
        self.source_images = torch.stack(self.source_images)
        self.target_images = torch.stack(self.target_images)
        
        if self.use_txt_features == "one_hot":
            self.one_hot_features = torch.stack(self.one_hot_features)
        elif self.use_txt_features == "categorical":
            self.cat_features = torch.stack(self.cat_features)
        elif self.use_txt_features == "histogram":
            self.source_histograms = torch.stack([h[0] for h in self.histograms])
            self.target_histograms = torch.stack([h[1] for h in self.histograms])
        
        print("Images and features preloaded and stored in GPU memory.")

    def precompute_features(self):
        print("Precomputing BERT and CLIP features...")
        self.bert_features = []
        self.clip_features = []
        
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        bert_model = BertModel.from_pretrained('bert-base-uncased').to(self.device).eval()
        clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(self.device).eval()
        
        for img_name in tqdm(self.img_files):
            # Precompute BERT features
            feature_text = " ".join(self.features[img_name])
            inputs = tokenizer(feature_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():              
                outputs = bert_model(**inputs)
            bert_features = outputs.last_hidden_state[:, 0, :].squeeze(0)  # Shape: (768,)
            self.bert_features.append(bert_features)

            # Precompute CLIP features
            image = self.source_images[len(self.bert_features) - 1].cpu()  # Get the corresponding preloaded image
            clip_inputs = clip_processor(images=image, return_tensors="pt").to(self.device)
            with torch.no_grad():           
                clip_features = clip_model.get_image_features(**clip_inputs)
            self.clip_features.append(clip_features.squeeze(0))  # Shape: (512,)
        
        self.bert_features = torch.stack(self.bert_features).to(self.device)
        self.clip_features = torch.stack(self.clip_features).to(self.device)
        
        del bert_model, tokenizer, clip_model, clip_processor
        print("BERT and CLIP features precomputed and stored in GPU memory.")

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        if self.pre_load_images:
            source = self.source_images[idx]
            target = self.target_images[idx]
        else:
            img_name = self.img_files[idx]
            source = read_image(os.path.join(self.IMGS_PATH, 'input', img_name))
            target = read_image(os.path.join(self.IMGS_PATH, 'target', img_name))
            
            if self.resize:
                source = F.resize(source, (self.image_size, self.image_size), interpolation=F.InterpolationMode.BICUBIC)
                target = F.resize(target, (self.image_size, self.image_size), interpolation=F.InterpolationMode.BICUBIC)
            
            source = source.to(self.device)
            target = target.to(self.device)

        if self.augment_data:
            if torch.rand(1).item() > 0.5:
                source = F.hflip(source)
                target = F.hflip(target)
            if torch.rand(1).item() > 0.5:
                source = F.vflip(source)
                target = F.vflip(target)

        if not self.use_txt_features:
            return source, target
        elif self.use_txt_features == "one_hot":
            if self.pre_load_images:
                return source, self.one_hot_features[idx], target
            else:
                one_hot = self.mlb.transform([self.features[self.img_files[idx]]])[0]
                return source, torch.tensor(one_hot, dtype=torch.float32, device=self.device), target
        elif self.use_txt_features == "categorical":
            if self.pre_load_images:
                return source, self.cat_features[idx], target
            else:
                cat = [self.feature_to_idx[cat][feat] for cat, feat in zip(self.feature_categories, self.features[self.img_files[idx]])]
                return source, torch.tensor(cat, dtype=torch.long, device=self.device), target
        elif self.use_txt_features == "embedded":
            return source, self.bert_features[idx], self.clip_features[idx], target
        elif self.use_txt_features == "histogram":
            if self.pre_load_images:
                return source, self.source_histograms[idx], target, self.target_histograms[idx]
            else:
                source_hist = self.compute_histogram(source).to(self.device)
                target_hist = self.compute_histogram(target).to(self.device)
                return source, source_hist, target, target_hist
        else:
            raise ValueError("Invalid value for use_txt_features. Must be False, 'one_hot', 'categorical', 'embedded', or 'histogram'.")

    def collate_fn(self, batch):
        if self.use_txt_features == "embedded":
            sources, bert_features, clip_features, targets = zip(*batch)
            return torch.stack(sources), torch.stack(bert_features), torch.stack(clip_features), torch.stack(targets)
        elif self.use_txt_features == "histogram":
            sources, source_hists, targets, target_hists = zip(*batch)
            return torch.stack(sources), torch.stack(source_hists), torch.stack(targets), torch.stack(target_hists)
        else:
            sources, features, targets = zip(*batch)
            return torch.stack(sources), torch.stack(features) if features[0] is not None else None, torch.stack(targets)

In [ ]:
f=FiveKDataset(64, mode="train", resize=True, augment_data=False, use_txt_features="histogram", device='cuda', pre_load_images=True)

In [ ]:
f[0][1].shape

In [ ]:
64*3

In [1]:
import os
import torch
from PIL import Image
import time
from torchvision.io import read_image
img_name = "a0002-dgw_005.jpg"
original_image = read_image(f"../dataset/FiveK/train/input/{img_name}")/255.0
target_image = read_image(f"../dataset/FiveK/train/target/{img_name}")/255.0
enhanced = torch.flatten(
    original_image.clone(), start_dim=0, end_dim=-1
)
target = torch.flatten(target_image.clone(), start_dim=0, end_dim=-1)

rmse = enhanced - target
rmse = torch.pow(rmse, 2).mean()
rmse = torch.sqrt(rmse)
psnr = (20 * torch.log10(1 / rmse))
rewards = psnr
print(rewards)
import os
os.chdir('..')

tensor(19.7316)


In [4]:
from src.envs.new_edit_photo import PhotoEditor
original_image = read_image("/home/zakaria/workspace/ai-photo-enhancer/src/sample_images/a0002-dgw_005.jpg")/255.0
device = 'cpu'
source_image = original_image.to(device).permute(1,2,0).unsqueeze(0)
parameters = torch.rand((1,10),device=device)
sliders = ['temp', 'tint', 'vibrance', 'saturation', 'contrast', 'exposure', 'shadows', 'highlights', 'whites', 'blacks']
photo_editor = PhotoEditor(sliders=sliders)

In [ ]:
original_image.shape

In [4]:
start_time = time.time()
enhanced = photo_editor(source_image,parameters)
end_time = time.time()
print(end_time-start_time)

9.531782865524292


In [5]:
import cProfile
import pstats

def run_enhancement():
    original_image = read_image("/home/zakaria/workspace/ai-photo-enhancer/src/sample_images/a0002-dgw_005.jpg")/255.0
    device = 'cpu'
    source_image = original_image.to(device).permute(1,2,0).unsqueeze(0)
    parameters = torch.rand((1,10),device=device)
    sliders = ['temp', 'tint', 'vibrance', 'saturation', 'contrast', 'exposure', 'shadows', 'highlights', 'whites', 'blacks']
    photo_editor = PhotoEditor(sliders=sliders)
    enhanced = photo_editor(source_image,parameters)

cProfile.run('run_enhancement()', 'enhancement_stats')

# Print sorted stats
p = pstats.Stats('enhancement_stats')
p.sort_stats('cumulative').print_stats(20)  # Print top 20 time-consuming functions

Sat Aug 10 18:24:20 2024    enhancement_stats

         373 function calls in 10.100 seconds

   Ordered by: cumulative time
   List reduced from 100 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   10.100   10.100 {built-in method builtins.exec}
        1    0.025    0.025   10.100   10.100 <string>:1(<module>)
        1    0.085    0.085   10.075   10.075 /tmp/ipykernel_147446/2466602610.py:4(run_enhancement)
        1    0.231    0.231    9.921    9.921 /home/zakaria/workspace/ai-photo-enhancer/src/envs/new_edit_photo.py:699(__call__)
        1    2.749    2.749    3.096    3.096 /home/zakaria/workspace/ai-photo-enhancer/src/envs/new_edit_photo.py:617(__call__)
        1    1.678    1.678    2.540    2.540 /home/zakaria/workspace/ai-photo-enhancer/src/envs/new_edit_photo.py:365(__call__)
        1    0.795    0.795    1.382    1.382 /home/zakaria/workspace/ai-photo-enhancer/src/envs/new_edit_photo.py

In [ ]:
rmse = enhanced - source_image
rmse = torch.pow(rmse, 2).mean()
rmse = torch.sqrt(rmse)
psnr = (20 * torch.log10(1 / rmse))
psnr 